In [ ]:
from __future__ import print_function

# Basic Spark 

We have discussed some [basic features of Spark](Spark_intro.ipynb) -- now we'll try to actually use the framework for some basic operations. 

In particular, this notebook will walk you through some of the basic [Spark RDD methods](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD). As you'll see, there is a lot more to it than `map` and `reduce`.

We will explore the concept of "lineage" in Spark RDDs and construct some simple key-value pair RDDs to write our first Spark applications.

If you need a reminder of some of the python concepts discussed earlier, you can make use of the [intro notebook](../intro/Spark_workshop_introduction.ipynb).

## Starting up the `SparkContext` locally

The most lightweight way of playing around with Spark is to run the whole Spark runtime on a single (local) machine. 

First, we need to do a few lines of setup (we can later move these to a startup script of some sort) and then we start the `SparkContext`

In [ ]:
import findspark
findspark.init()
import pyspark

In [ ]:
sc = pyspark.SparkContext(master='local[*]')
print(sc)

Hurrah! We have a Spark Context! Now lets get some data into the Spark universe: 

In [ ]:
data = xrange(100)
data_rdd = sc.parallelize(data)
print('Number of elements: ', data_rdd.count())
print('Sum and mean: ', data_rdd.sum(), data_rdd.mean())

Now if you look at your console, you will see *a lot* of output -- Spark is reporting all the stages of execution and can become rather verbose. Initially it's useful to inspect this output just to see what's going on and to see when issues arise. Later on we'll see how to quiet it down. 

In addition, each Spark application runs its own dedicated Web UI, accessible by default at `driver:4040`. In this case this is http://localhost:4040. 

This gives you a lot of nice information about the state of your job, including stats on execution time of individual tasks, available memory on all of the workers, links to worker logs, etc. You will probably begin to appreciate some of this information when things start to go wrong...

## Map/Reduce 

Lets bring some of the simple python-only examples from the [first notebook]('../intro/Spark_workshop_Introduction.ipynb) into the Spark framework. The first map function we made was simply doubling the input array, so lets do this here. 

Write the function `double_the_number` and then use this function with the `map` method of `data_rdd` to yield `double_rdd`:

In [ ]:
def double_the_number(x) : 
    return x*2

In [ ]:
help(data_rdd.map)

In [ ]:
double_rdd = data_rdd.<FILL>

Not much happened here - or at least, no tasks were launched (you can check the console and the Web UI). Spark simply recorded that the `data_rdd` maps into `double_rdd` via the `map` method using the `double_the_number` function. You can see some of this information by inspecting the RDD debug string: 

In [ ]:
print(double_rdd.toDebugString())

In [ ]:
# comparing the first few elements of the original and mapped RDDs using take
print(data_rdd.take(10))
print(double_rdd.<FILL>)

Now if you go over to check on the [stages in the Spark UI](http://localhost:4040/stages/) you'll see that jobs were run to grab data from the RDD. In this case, a single task was run since all the numbers needed reside in one partition. Here we used `take` to extract a few RDD elements, a very very very convenient method for checking the data inside the RDD and debugging your map/reduce operations. 

Often, you will want to make sure that the function you define executes properly on the whole RDD. The most common way of forcing Spark to execute the mapping on all elements of the RDD is to invoke the `count` method: 

In [ ]:
double_rdd.count()

If you now go back to the [stages page](http://localhost:4040/stages), you'll see that four tasks were run for this stage. 

In our initial example of using `map` in pure python code, we also used an inline lambda function. For such a simple construct like doubling the entire array, the lambda function is much neater than a separate function declaration. This works exactly the same way here.

Map the `data_rdd` to `double_lambda_rdd` by using a lambda function to multiply each element by 2: 

In [ ]:
double_lambda_rdd = data_rdd.<FILL>
print(double_lambda_rdd.take(10))

Finally, do a simple `reduce` step, adding up all the elements of `double_lambda_rdd`:

In [ ]:
from operator import add
double_lambda_rdd.<FILL>

(Spark RDDs actually have a `sum` method which accomplishes essentially the same thing)

## Filtering

A critical step in many analysis tasks is to filter down the input data. In Spark, this is another *transformation*, i.e. it takes an RDD and maps it to a new RDD via a filter function. The filter function needs to evaluate each element of the RDD to either `True` or `False`. 

Use `filter` with a lambda function to select all values less than 10: 

In [ ]:
filtered_rdd = data_rdd.filter(<FILL>)
filtered_rdd.count()

Of course we can now apply the `map` and double the `filtered_rdd` just as before: 

In [ ]:
filtered_rdd.map(<FILL>).take(10)

Note that each RDD transformation returns a new RDD instance to the caller -- for example:

In [ ]:
data_rdd.filter(lambda x: x % 2)

You can therefore string together many transformations without creating a separate instance variable for each step. Our `filter` + `map` step can therefore be combined into one. Note that if we surround the operations with "( )" we can make the code more readable by placing each transformation on a separate line: 

In [ ]:
composite = (data_rdd.filter(lambda x: x % 2)
                     .map(lambda x: x*2))

Again, if you now look at the [Spark UI](http://localhost:4040) you'll see that nothing actually happened -- no job was trigerred. The `composite` RDD simply encodes the information needed to create it. 

If an action is executed that only requires a part of the RDD, only those parts will be computed. If we cache the RDD and only calculate a few of the elements, this will be made clear:

In [ ]:
composite.cache()
composite.take(10)

If you look at the [storage information](http://localhost:4040/storage/) you'll see that just a quarter of the RDD is cached. Now if we trigger the full calculation, this will increase to 100%:

In [ ]:
composite.count()

## Key, value pair RDDs

`key`,`value` pair data is the "bread and butter" of map/reduce programming. Think of the `value` part as the meat of your data and the `key` part as some crucial metadata. For example, you might have time-series data for CO$_2$ concentration by geographic location: the `key` might be the coordinates or a time window, and `value` the CO$_2$ data itself. 

If your data can be expressed in this way, then the map/reduce computation model can be very convenient for pre-processing, cleaning, selecting, filtering, and finally analyzing your data. 

Spark offers a `keyBy` method that you can use to produce a key from your data. In practice this might not be useful often but we'll do it here just to make an example: 

In [ ]:
# key the RDD by x modulo 5
keyed_rdd = data_rdd.keyBy(lambda x: x%5)

In [ ]:
keyed_rdd.take(20)

This created keys with values 0-4 for each element of the RDD. We can now use the multitude of `key` transformations and actions that the Spark API offers. For example, we can revisit `reduce`, but this time do it by `key`: 

## `reduceByKey`

In [ ]:
# use the add operator in the `reduceByKey` method
red_by_key = keyed_rdd.<FILL>
red_by_key.collect()

Unlike the global `reduce`, the `reduceByKey` is a *transformation* --> it returns another RDD. Often, when we reduce by key, the dataset size is reduced enough that it is safe to pull it completely out of Spark and into the driver (i.e. this notebook). A useful way of doing this is to automatically convert it to python dictionary for subsequent processing with the `collectAsMap` method:

In [ ]:
red_dict = red_by_key.collectAsMap()
red_dict

In [ ]:
# access by key
red_dict[0]

## `groupByKey`

If you want to collect the elements belonging to a key into a list in order to process them further, you can do this with `groupByKey`. Note that if you want to group the elements only to do a subsequent reduction, you are far better off using `reduceByKey`, because it does the reduction locally on each partition first before communicating the results to the other nodes. By contrast, `groupByKey` reshuffles the entire dataset because it has to group *all* the values for each key from all of the partitions. 

In [ ]:
keyed_rdd.groupByKey().collect()

Note the ominous-looking `pyspark.resultiterable.Resultiterable`: this is exactly what it says, an iterable. You can turn it into a list or go through it in a loop. For example:

In [ ]:
key, iterable = keyed_rdd.groupByKey().first()

In [ ]:
list(iterable)

In [ ]:
# print out every value in the iterable
for <FILL>

## `sortBy`

Use the `sortBy` method of `red_by_key` to return a list sorted by the sums and print it out. 

In [ ]:
sorted_red = red_by_key.sortBy(<FILL>).collect()

In [ ]:
assert(sorted_red == [(4, 1030), (3, 1010), (2, 990), (1, 970), (0, 950)])

Finally, to shut down the `SparkContex`, call `sc.stop()`:

In [ ]:
sc.stop()